In [1]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd



In [2]:
train_xml = "train/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml"

predator_txt_train = "train/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt"
predator_txt_1 = "train/pan12-sexual-predator-identification-diff.txt"

test_xml = "test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml"
predator_txt_test = "train/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt"


In [3]:
#TRAIN DATA
# Parsing the XML file and getting the training data\n",
tree = ET.parse(train_xml)
root = tree.getroot()

# Extracting the data and converting to dataframe from XML
convo_id_csv = []
author_id_csv = []
text_csv = []
time_csv = []
s = []

c = 0
for convo in root:
    convo_id = convo.attrib['id']
    for msg in convo:
        author_id = msg[0].text
        time = msg[1].text
        text = msg[2].text

        convo_id_csv.append(convo_id)
        author_id_csv.append(author_id)
        text_csv.append(text)
        time_csv.append(time)
        s.append(0)

    c+=1
    
csv_dic = {
    "conversation id": convo_id_csv,
    "author id": author_id_csv,
    "msg" : text_csv,
    "time":time_csv,
    "predator": s
}


df_initial = pd.DataFrame.from_dict(csv_dic)
df_initial.head()

########################################################

# Predator messages segration
temp_array = []

f = open(predator_txt_train, "r")
predator_ids = f.readlines()
f.close()

for i in predator_ids:
    #print(i.split("\n")[0])
    temp_array.append(i.split("\n")[0])
predator_ids = temp_array

f = open(predator_txt_1, "r")
predator_ids_1 = f.readlines()
f.close()

for i in predator_ids_1:
    #print(i.split("\t")[0])
    temp_array.append(i.split("\t")[0])
predator_ids_1 = temp_array

####################################################


# updating predator as 1 in csv
for i in range(0,len(author_id_csv)):
    if (author_id_csv[i] in predator_ids):
        s[i] = 1
        

csv_dic = {
    "conversation id": convo_id_csv,
    "author id": author_id_csv,
    "msg" : text_csv,
    "time":time_csv,
    "predator": s
}


df_predator_update = pd.DataFrame.from_dict(csv_dic)

# Removing rows which have null msg
df_predator_update["msg"].fillna(value="hello", inplace=True)

#################################################

# 900000+ rows
df_predator_update.to_csv('train_csv.csv')
#df_predator_update.to_excel('train.xlsx', engine='xlsxwriter')  

       
        

In [4]:
#TEST DATA
# Parsing the XML file and getting the training data\n",
tree = ET.parse(test_xml)
root = tree.getroot()

# Extracting the data and converting to dataframe from XML
convo_id_csv = []
author_id_csv = []
text_csv = []
time_csv = []
s = []

c = 0
for convo in root:
    convo_id = convo.attrib['id']
    for msg in convo:
        author_id = msg[0].text
        time = msg[1].text
        text = msg[2].text

        convo_id_csv.append(convo_id)
        author_id_csv.append(author_id)
        text_csv.append(text)
        time_csv.append(time)
        s.append(0)

    c+=1
    
csv_dic = {
    "conversation id": convo_id_csv,
    "author id": author_id_csv,
    "msg" : text_csv,
    "time":time_csv,
    "predator": s
}


df_initial = pd.DataFrame.from_dict(csv_dic)
df_initial.head()

########################################################

# Predator messages segration
temp_array = []

f = open(predator_txt_test, "r")
predator_ids = f.readlines()
f.close()

for i in predator_ids:
    #print(i.split("\n")[0])
    temp_array.append(i.split("\n")[0])
predator_ids = temp_array

####################################################


# updating predator as 1 in csv
for i in range(0,len(author_id_csv)):
    if (author_id_csv[i] in predator_ids):
        s[i] = 1
        

csv_dic = {
    "conversation id": convo_id_csv,
    "author id": author_id_csv,
    "msg" : text_csv,
    "time":time_csv,
    "predator": s
}


df_predator_update = pd.DataFrame.from_dict(csv_dic)

# Removing rows which have null msg
df_predator_update["msg"].fillna(value="hello", inplace=True)

#################################################

# 900000+ rows
df_predator_update.to_csv('test_csv.csv')


       
        


In [5]:
#######################################################
import pandas as pd
df_all = pd.read_csv('train_csv.csv')

In [6]:

def find_url(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url] 

###############################################################
# droppin na values
df_all.dropna(subset = ['msg'], inplace = True)
df_all.index = range(0, df_all.shape[0])
df_all.info()

# extracting url and replacing it with the word url
df_url = df_all[df_all['chat_message'].str.contains(r'https')]
#df_url.index = range(0, df_url.shape[0])

df_all[df_all['conversation id'] == df_url['conversation id'][1143]]
urls = find_url(df_url['msg'][1143])

re.sub(regex, 'url', df_url['msg'][903259])
df_url['msg'] = df_url['msg'].apply(lambda x: re.sub(regex, 'url', x))
df_all['msg'][243] = df_url['msg'][243]
for ind in list(df_url.index):
    df_all['msg'][ind] = df_url['msg'][ind]

print(df)
#df_all.to_csv('train_nourl.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903604 entries, 0 to 903603
Data columns (total 6 columns):
Unnamed: 0         903604 non-null int64
conversation id    903604 non-null object
author id          903604 non-null object
msg                903604 non-null object
time               903604 non-null object
predator           903604 non-null int64
dtypes: int64(2), object(4)
memory usage: 41.4+ MB


NameError: name 'df_url' is not defined